# Imports

In [1]:
import torch
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
import json
import numpy as np

In [2]:
import sys

sys.path.append("../../resource/PiDiNet/pidinet-master")
sys.path.append("../../src")

from models.pidinet import PiDiNet
from dataset import BIPEDv2, transforms

# Prepare